In [104]:
NOTE_DATA_DIR = '/local/XW/DATA/MIMIC/noteevents_by_sid/'

In [105]:
import os,sys, re
import numpy as np
import pandas as pd
from tqdm import tqdm
import cPickle as pk
np.random.seed(1) # to be reproductive


In [106]:
from stop_words import get_stop_words
en_stop = get_stop_words('en')
en_stop = set(en_stop)

from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+|_') # underscore can't be matched by \W+

from nltk.stem.porter import PorterStemmer
p_stemmer = PorterStemmer()

import gensim
from gensim import corpora, models

In [107]:
notes = []
for fn in os.listdir(NOTE_DATA_DIR):
    fpath = os.path.join(NOTE_DATA_DIR, fn)
    df = pd.read_csv(fpath)    
    note = '\n'.join(df.text).lower()
    notes.append(note)

In [108]:
# filters=r'''!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n+0123456789'''
# re.sub(filters, '', note)

In [109]:
texts = []
for raw_note in notes:
    no_stop = filter( lambda tk: tk not in en_stop, tokenizer.tokenize(raw_note) )
    filtered = filter(lambda tk: pat.match(tk) is None and len(tk)>1, no_stop)
    stemmed = map(lambda tk: p_stemmer.stem(tk), filtered)
    texts.append( stemmed )

In [110]:
print texts[0]

[u'nurs', u'transfer', u'note', u'pt', u'admit', u'nicu', u'sepsi', u'eval', u'pleas', u'see', u'attend', u'note', u'detail', u'regard', u'matern', u'histori', u'deliveri', u'detail', u'infant', u'stabl', u'ra', u'rr', u'sat', u'ls', u'clear', u'retract', u'note', u'hr', u'murmur', u'infant', u'well', u'perfus', u'bw', u'cbc', u'bc', u'sent', u'pend', u'time', u'infant', u'sepsi', u'abx', u'amp', u'gent', u'piv', u'place', u'left', u'hand', u'med', u'administ', u'order', u'stick', u'infant', u'stabl', u'transfer', u'nbn', u'continu', u'monitor', u'sepsi', u'neonatolog', u'attend', u'triag', u'note', u'babi', u'name', u'ni', u'known', u'patient', u'lastnam', u'term', u'male', u'infant', u'admit', u'nicu', u'sepsi', u'evalu', u'ask', u'evalu', u'babi', u'dr', u'first', u'name4', u'namepattern1', u'last', u'name', u'namepattern1', u'mother', u'year', u'old', u'g1', u'p0', u'pn', u'po', u'ab', u'neg', u'hbsag', u'neg', u'rpr', u'nr', u'ri', u'gb', u'neg', u'pregnanc', u'uncompl', u'deliver

In [ ]:
dictionary = corpora.Dictionary(texts)

In [ ]:
corpus = [dictionary.doc2bow(text) for text in texts] # convert to (token_id, token_cnt) tuple

In [ ]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=300, id2word = dictionary, passes=20)

In [ ]:
ldamodel.print_topics(20)